### Embed data using per signal LSTMs

Embed the downstream prediction data using the per signal LSTMs

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from phase.embedding import *

PATH = os.path.expanduser("~/upstream_embedding/")

# The upstream task we trained the LSTMs with
task = "nextfive"
# task = "minimum5"
# task = "hypo"

# Downstream tasks corresponding to different data sets that we will emebd 
final_tasks = ["etco235", "desat_bool92_5_nodesat", "nibpm60"]

for final_task in final_tasks:
    for hosp_data in [0,1]:
        for hosp_model in [0,1]:
            for i in range(len(X_lst)):
                # Find the minimum val_loss model
                feat = X_lst[i]; feat_id = feat_ids[i]

                # Load min_mod
                MPATH = "{}/models/200epochs/{}/hospital_{}/".format(PATH,task,hosp_model)
                min_mod = load_min_model(MPATH,feat)

                # Make the model without the final dense layer
                model, _, _ = create_model(output_size=False,min_mod=min_mod)

                # Load original data hospital0 - Corresponds to the feature the LSTM was trained on
                DPATH = PATH+"data/{}/hospital{}_data/proc/".format(final_task,hosp_data)
                X_trval,X_test1 = load_trval_test_data(DPATH,feat)

                # Process and save
                SPATH = PATH+"data/{}/hospital{}_data/".format(final_task,hosp_data)
                SPATH += "hidden200/{}/model_{}/".format(task,hosp_model)
                suffix = "embedding_data{}_model{}.npy".format(hosp_data,hosp_model)
                embed_and_save(SPATH,suffix,model,X_trval,X_test1,feat,task)